In [1]:
!pip install langchain faiss-cpu tiktoken


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [19]:
from langchain_groq import ChatGroq
from langchain.document_loaders import TextLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS 
from langchain_text_splitters import RecursiveCharacterTextSplitter 
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import (
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from langchain_core.output_parsers import StrOutputParser
import chainlit as cl
from langchain.memory import ConversationBufferMemory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.chains import (
    StuffDocumentsChain, LLMChain, ConversationalRetrievalChain
)


In [7]:
def load_and_split_documents():
    docs = []
    
    # Load documents from the "docs" directory
    for file in os.listdir("docs"):
        if file.endswith(".txt"):
            loader = TextLoader(os.path.join("docs", file))
            docs.extend(loader.load())
    
    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=50)
    splits = text_splitter.split_documents(docs)
    
    return splits

In [9]:
splits = load_and_split_documents()

In [10]:

embeddings = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")

def initialize_vectorstore(splits):
    return FAISS.from_documents(documents=splits, embedding=embeddings)

db = initialize_vectorstore(splits)

# Save to disk
db.save_local("faiss_index")


/home/codespace/.python/current/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-05-30 09:21:06 - Use pytorch device_name: cpu
2025-05-30 09:21:06 - Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2025-05-30 09:21:11 - Loading faiss with AVX2 support.
2025-05-30 09:21:11 - Successfully loaded faiss with AVX2 support.
2025-05-30 09:21:11 - Failed to load GPU Faiss: name 'GpuIndexIVFFlat' is not defined. Will not load constructor refs for GPU indexes. This is only an error if you're trying to use GPU Faiss.


In [11]:
db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
retriever = db.as_retriever()


In [12]:
api_key = os.getenv("GROQ_AP_KEY")
llm = ChatGroq(groq_api_key = api_key ,  model_name = "Gemma2-9b-It" , temperature = 0)

In [13]:
prompt_template = """You are an expert in immigration assistance powered by AI. Your role is to answer questions related to visa processes and the services offered by VisaBridge. 
Please consider the following context and user question:
Context: {context}
Question: {question}
"""

In [14]:
prompt = ChatPromptTemplate.from_template(prompt_template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
rag_chain = (
    {"context": retriever| format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [15]:
rag_chain.invoke("How long does the visa application process take?")

2025-05-30 09:21:23 - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


"That's a great question!  Unfortunately, there's no one-size-fits-all answer to how long a visa application takes. \n\nIt really depends on a few factors:\n\n* **The specific visa type:**  Tourist visas generally have shorter processing times than work or student visas, for example.\n* **The destination country:**  Each country has its own processing times, and some are known to be faster than others.\n* **Time of year:**  Visa application volumes can fluctuate depending on the season, which can impact processing times.\n* **Completeness of your application:**  Providing all required documents and information accurately from the start can help speed up the process.\n\n**Here's where VisaBridge can help!** We can provide you with more specific information about the estimated processing time for your desired visa based on your individual circumstances.  \n\nJust tell us:\n\n* **What type of visa are you applying for?**\n* **Where are you applying to travel?**\n\nWe're here to guide you 

In [20]:
@cl.on_chat_start
async def on_chat_start():
    retriever = retriever()
    message_history = ChatMessageHistory()
    memory = ConversationBufferMemory(
        memory_key="chat_history",
        output_key="answer",
        chat_memory=message_history,
        return_messages=True,
    )
    chain = ConversationalRetrievalChain.from_llm(
        llm,
        chain_type="stuff",
        retriever=retriever,
        memory=memory,
        return_source_documents=True,
    )
    cl.user_session.set("chain", chain)

In [21]:
@cl.on_message
async def main(message: cl.Message):
    chain = cl.user_session.get("chain")
    cb = cl.AsyncLangchainCallbackHandler()
    res = await chain.acall(message.content, callbacks=[cb])
    answer = res["answer"]
    source_documents = res["source_documents"]
    text_elements = []
    if source_documents:
        for source_idx, source_doc in enumerate(source_documents):
            source_name = f"source_{source_idx}"
            
            text_elements.append(
                cl.Text(content=source_doc.page_content, name=source_name)
            )
        source_names = [text_el.name for text_el in text_elements]
        if source_names:
            answer += f"\nSources: {', '.join(source_names)}"
        else:
            answer += "\nNo sources found"
    await cl.Message(content=answer, elements=text_elements).send()

In [14]:
def chat():
    print("VisaBridge Assistant. Type 'exit' to stop.")
    while True:
        query = input("You: ")
        if query.lower() == 'exit':
            break
        result = qa_chain.invoke(query)
        print("Bot:", result)

if __name__ == "__main__":
    chat()


VisaBridge Assistant. Type 'exit' to stop.


ValueError: Got multiple output keys: dict_keys(['answer', 'source_documents']), cannot determine which to store in memory. Please set the 'output_key' explicitly.

In [16]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

# Assuming llm and retriever are defined
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    return_source_documents=True,
    output_key="answer"  # Specify the key to store in memory
)

query = "What document should I provide?"
result = qa_chain.invoke(query, output_key="answer")  # Specify which key to use

# Print the result
print(result)

ValueError: Got multiple output keys: dict_keys(['answer', 'source_documents']), cannot determine which to store in memory. Please set the 'output_key' explicitly.

In [ ]:
import logging

logging.basicConfig(filename="chat_log.txt", level=logging.INFO)

def chat():
    print("VisaBridge Assistant. Type 'exit' to stop.")
    while True:
        query = input("You: ")
        if query.lower() == 'exit':
            break
        result = qa_chain.run(query)
        logging.info(f"You: {query}")
        logging.info(f"Bot: {result}")
        print("Bot:", result)
